# <b>[Colaboratory]Tensorflow2-controller.ipynb</b><br>
このノートブックはTensorflow2 Object Detection APIの学習/推論のハンズオン用スクリプトです<br><br>
Colaboratoryのハードウェア アクセラレータ設定をGPUにして実行してください<br><br>
Github URL : [Tensorflow2-contoroller](https://github.com/tattuko/Tensorflow2-controller.git)



# <b>Google Driveマウント</b>
※checkpoint、saved model格納先

In [ ]:
from google.colab import drive
drive.mount('./gdrive')

# <b>Tensorflow Object Detection API設定</b>

### Protocol Buffers

In [ ]:
!curl -OL https://github.com/google/protobuf/releases/download/v3.2.0/protoc-3.2.0-linux-x86_64.zip
!unzip protoc-3.2.0-linux-x86_64.zip -d protoc3
!sudo mv protoc3/bin/* /usr/local/bin/
!sudo mv protoc3/include/* /usr/local/include/
!rm -rf protoc3 protoc-3.2.0-linux-x86_64.zip

In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models
%cd /content/models/research

!/usr/local/bin/protoc object_detection/protos/*.proto --python_out=.

### 必要ライブラリインストール

In [ ]:
!cp /content/models/research/object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
# インストール成否確認(Confirmation of successful installation)
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

# <b>Tensorflow2-controllerリポジトリクローン</b>

In [ ]:
!git clone https://github.com/tattuko/Tensorflow2-controller.git

# <b>TFRecordをアップロード</b>
「Tensorflow2-ObjectDetectionAPI-controller/02_tfrecord」にVoTTからエクスポートしたTFRecordとtf_label_map.pbtxtを格納してください。
<br><br>
![](https://user-images.githubusercontent.com/37477845/94039064-31f8cc80-fe02-11ea-81f2-28427d759099.png)


# <b>学習データ/検証データ 分割(Split Training data/validation data)</b>

In [ ]:
original_data_dir = '/content/models/research/Tensorflow2-controller/02_tfrecord'
train_data_dir = '/content/models/research/train_data'
val_data_dir = '/content/models/research/val_data'

In [ ]:
# ディレクトリ作成(Create a directory)
import os
import shutil

shutil.rmtree(train_data_dir, ignore_errors=True)
shutil.rmtree(val_data_dir, ignore_errors=True)
os.mkdir(train_data_dir)
os.mkdir(val_data_dir)

In [ ]:
import glob

# ファイル数カウント(Count the number of files)
file_count = len(glob.glob(original_data_dir + '/*.tfrecord'))
print('File count : ' + str(file_count))

In [ ]:
import random

# 学習データ/検証データ 分割(Split Training data/validation data.)
train_ratio = 0.75

file_list = glob.glob(original_data_dir + '/*.tfrecord')
random_sample_list = random.sample(file_list, file_count)

# ディレクトリへコピー(Copy to directory)
for index, filepath in enumerate(random_sample_list):
    if index < int(file_count * train_ratio):
        # 学習データ(Training data)
        shutil.copy2(filepath, train_data_dir)
    else:
        # 検証データ(Validation data)
        shutil.copy2(filepath, val_data_dir)

# <b>学習済モデル(Pre-Trained model)</b>

このハンズオンでは「Tensorflow2-controller/03_pretrained_model」に学習済モデル(EfficientDet-D0)が格納してあります。<br><br><br>学習済モデル取得元：http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz

# <b>パイプラインコンフィグアップロード(Pipeline config upload)</b>

パイプラインコンフィグを修正し「Tensorflow2-controller/03_pretrained_model」にアップロードしてください<br><br>
パイプラインコンフィグは以下の行を修正します<br>

* 3行目(Line 3)：クラス数(num_classes)<br>変更前(Before) : 90<br>変更後(After) : 1<br>
* 134行目(Line 134)：バッチサイズ(batch_size)<br>変更前(Before) : 128<br>変更後(After) : 16<br>
* 161行目(Line 161)：ファインチューニング用のチェックポイント格納先(fine_tune_checkpoint)<br>変更前(Before) : "PATH_TO_BE_CONFIGURED"<br>変更後(After) : "/content/models/research/Tensorflow2-controller/03_pretrained_model/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0"
* 167行目(Line 167)：ファインチューニング方法(fine_tune_checkpoint_type)<br>変更前(Before) : "classification"<br>変更後(After) : "detection"<br>
* 168行目(Line 168)：Googleカスタム 16ビットbrain浮動小数点の使用有無(use_bfloat16)<br>変更前(Before) : true<br>変更後(After) : false<br>
* 172行目(Line 172)：ラベルマップファイルの格納先(label_map_path)<br>変更前(Before) : "PATH_TO_BE_CONFIGURED/label_map.txt"<br>変更後(After) : "/content/models/research/Tensorflow2-controller/02_tfrecord/tf_label_map.pbtxt"<br>
* 174行目(Line 174)：学習データの格納先(input_path)<br>変更前(Before) : "PATH_TO_BE_CONFIGURED/train2017-?????-of-00256.tfrecord"<br>変更後(After) : "/content/models/research/train_data/??????.tfrecord"<br>
* 185行目(Line 185)：ラベルマップファイルの格納先(label_map_path)<br>変更前(Before) : "PATH_TO_BE_CONFIGURED/label_map.txt"<br>変更後(After) : "/content/models/research/Tensorflow2-controller/02_tfrecord/tf_label_map.pbtxt"<br>
* 189行目(Line 189)：バリデーションデータの格納先(input_path)<br>変更前(Before) : "PATH_TO_BE_CONFIGURED/val2017-?????-of-00032.tfrecord"<br>変更後(After) : "/content/models/research/val_data/??????.tfrecord"
<br><br>
![](https://user-images.githubusercontent.com/37477845/94040113-83558b80-fe03-11ea-8d8a-a5304efcca1d.png)


# <b>モデル訓練(Model training)</b>

### Googleドライブに保存先ディレクトリを作成

In [ ]:
!mkdir '/content/gdrive/My Drive/Tensorflow2-controller'

### TensorBoard

In [ ]:
%load_ext tensorboard

In [ ]:
tensorboard --logdir '/content/gdrive/My Drive/Tensorflow2-controller'

# <b>学習(Training)</b>

学習はColaboratory上で1000ステップにつき、約25分かかります

In [ ]:
!python object_detection/model_main_tf2.py \
    --pipeline_config_path="/content/models/research/Tensorflow2-controller/03_pretrained_model/efficientdet_d0_coco17_tpu-32/pipeline.config" \
    --model_dir="/content/gdrive/My Drive/Tensorflow2-controller" \
    --num_train_steps=1000 \
    --alsologtostderr \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps=100

# <b>saved model形式へエクスポート</b>

In [ ]:
!python object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path="/content/models/research/Tensorflow2-controller/03_pretrained_model/pipeline.config" \
    --trained_checkpoint_dir="/content/gdrive/My Drive/Tensorflow2-controller" \
    --output_directory="/content/gdrive/My Drive/Tensorflow2-controller/output"

# <b>モデルロード(Load model)</b>

In [ ]:
import tensorflow as tf

model_path = '/content/gdrive/My Drive/Tensorflow2-controller/output/saved_model'

DEFAULT_FUNCTION_KEY = 'serving_default'
loaded_model = tf.saved_model.load(model_path)
inference_func = loaded_model.signatures[DEFAULT_FUNCTION_KEY]

# <b>推論(Inference)</b>

In [ ]:
# 推論対象のテスト画像一覧(List of test images to be inferred)
import glob
import copy
import cv2
from google.colab.patches import cv2_imshow

test_data_dir = '/content/models/research/Tensorflow2-controller/04_test_data'
testfile_list = sorted(glob.glob(test_data_dir + '/*.jpg'))

In [ ]:
# 推論用関数(Function for inference)
def run_inference_single_image(image, inference_func):
    tensor = tf.convert_to_tensor(image)
    output = inference_func(tensor)

    output['num_detections'] = int(output['num_detections'][0])
    output['detection_classes'] = output['detection_classes'][0].numpy()
    output['detection_boxes'] = output['detection_boxes'][0].numpy()
    output['detection_scores'] = output['detection_scores'][0].numpy()
    return output

In [ ]:
import cv2
import numpy as np
from PIL import Image

filenames = []

# 動画書き出し用設定(VideoWriter setting)
temp_image = cv2.imread(testfile_list[0], cv2.IMREAD_UNCHANGED)
image_width, image_height = temp_image.shape[1], temp_image.shape[0]
fourcc = 'mp4v'
writer_fourcc = cv2.VideoWriter_fourcc(*fourcc)
videowriter = cv2.VideoWriter('result.mp4', writer_fourcc, 10, (image_width, image_height))

# 推論(Inference)
for filecount, testfile in enumerate(testfile_list):
    image = cv2.imread(testfile, cv2.IMREAD_UNCHANGED)
    debug_image = copy.deepcopy(image)

    image_width, image_height = image.shape[1], image.shape[0]
    image = image[:, :, [2, 1, 0]]  # BGR2RGB
    image_np_expanded = np.expand_dims(image, axis=0)

    output = run_inference_single_image(image_np_expanded, inference_func)

    num_detections = output['num_detections']
    for i in range(num_detections):
        score = output['detection_scores'][i]
        bbox = output['detection_boxes'][i]
        # class_id = output['detection_classes'][i].astype(np.int)

        if score < 0.85:
            continue

        x1, y1 = int(bbox[1] * image_width), int(bbox[0] * image_height)
        x2, y2 = int(bbox[3] * image_width), int(bbox[2] * image_height)

        # 推論結果描画(Inference result drawing)
        cv2.rectangle(debug_image, (x1, y1), (x2, y2), (255, 255, 255), 2)
        cv2.putText(debug_image, str('{:.2f}'.format(score)), (x1, y1-10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.rectangle(debug_image, (x1, y1), (x2, y2), (255, 255, 255), 2)
    videowriter.write(debug_image)
videowriter.release()

# <b>推論結果確認</b>

In [ ]:
import imageio
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

def play_video(video, interval=100):
    video = imageio.mimread(video)
    fig = plt.figure(figsize=(9, 6))

    movie = []
    for i in range(len(video)):
        img = plt.imshow(video[i], animated=True)
        plt.axis('off')
        movie.append([img])

    anime = animation.ArtistAnimation(fig, movie, interval=interval, repeat_delay=1000)
    plt.close()
    return anime

HTML(play_video('result.mp4').to_html5_video()) 